# Import the Necessary Libraries

In [1]:
# Data and Plotting
import pandas as pd
import numpy as np
import plotly.express as px
from ucimlrepo import fetch_ucirepo 
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, DetCurveDisplay

# Data encoding and Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, Normalizer
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# Dimensionality Reduction and Clustering Algorithms
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans

from sklearn.utils.class_weight import compute_sample_weight

# Data spliting and Cross Validation and Performance Metrics
from sklearn.model_selection import train_test_split as tts, StratifiedKFold, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from sklearn.metrics import recall_score, f1_score, balanced_accuracy_score, roc_auc_score
from sklearn.metrics import  make_scorer,  precision_score, accuracy_score

# models
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.neural_network import MLPClassifier as FNN

import pickle

%matplotlib notebook

# Load the Dataset

In [2]:
data = pd.read_csv("Thyroid_Diff.csv")

X = data.drop("Recurred", axis=1)
y = data["Recurred"]
y = y.map({"No":0,"Yes":1})

X_train, X_test, y_train, y_test = tts(X, y, 
                                       train_size=.75, 
                                       random_state=321
                                      )
y_train.value_counts()

Recurred
0    210
1     77
Name: count, dtype: int64

# Define the Steps that are Common to the Methods

## a. The Data Preprocessing Step

In [3]:
# Define preprocessor
num_features = list(X.columns[:1])
cat_features = list(X.columns[1:])

# Define the numerical transformer
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), 
    ("scaler", MinMaxScaler())
])

# Define the categorical transformer
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
])

# Define your ColumnTransformer (preprocessor)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ],
    remainder="passthrough",
)

# Compute sample weights based on class imbalance
sample_weights = compute_sample_weight(class_weight="balanced", y=y_train)

## b. Define the Models and Metrics

In [4]:
# Define the models
models = {
    "RF": RFC(),
    "GB": GBC(),
    "SVM": SVC(probability=True),
    "LR": LR(),
    "KNN": KNN(),
    "FNN": FNN(),
}

# Define the list of metrics for model evaluation on test set
metrics = {
    "Balanced Accuracy": balanced_accuracy_score,
    "F1 Score": f1_score,
    "ROC AUC": roc_auc_score,
    "Sensitivity": recall_score,
    "Specificity": recall_score,
    "Precision": precision_score
}

# Define the list of metrics for models evaluation using 10-fold CV
ten_fold_CV_metrics = {
    "Balanced Accuracy": "balanced_accuracy",
    "F1 Score": "f1_macro",
    "ROC AUC": "roc_auc",
    "Sensitivity": make_scorer(recall_score, pos_label=1),
    "Specificity":  make_scorer(recall_score, pos_label=0),
    "Precision": "precision"
}

# PCA-Based Classification Pipelines 

In [5]:
# These hyperparameters were tested before fine-tuning each one at finer granularity to improve model performance
param_grids = {
    "RF": {
        "clf__n_estimators": [400, 403],
        "clf__max_depth": [None],
        "clf__min_samples_split": [6, 7],
        "clf__min_samples_leaf": [4, 5],
        "clf__criterion": ["gini", "entropy", "log_loss"],
#         "clf__class_weight": [{0: 1, 1: 3}],
        "clf__max_features": ["log2"], 
    },
    "GB": {
        "clf__n_estimators": [150, 152],
        "clf__loss": ["exponential"],
        "clf__learning_rate": [0.35, 0.36],
        "clf__max_depth": [5, 6],
        "clf__criterion": ["squared_error", "friedman_mse",],
        "clf__tol": [1e-11, 1e-12]
    },
    "SVM": {
        "clf__C": [0.1, 0.12, 0.14],
        "clf__kernel": ["rbf", "sigmoid"],
        "clf__tol": [1e-1, 1e-2, 1e-3],
        
    },
    "LR": {
        "clf__C": [0.35, 0.36, 0.7],
        "clf__penalty": ["l1"],
        "clf__solver": ["liblinear"],

    },
    "KNN": {
        "clf__n_neighbors": [16, 17],
        "clf__weights": ["distance"],
        "clf__p": [2, 3, 4]
    },
    "FNN": {
        "clf__hidden_layer_sizes": [(300,), (130, 160)],
        "clf__max_iter": [10000, 12000],
        "clf__activation": ["relu"], 
        "clf__solver": ["adam", "sgd", "lbfgs"],
        "clf__learning_rate": ["adaptive", "constant"],
        "clf__early_stopping": [True],
        "clf__tol": [1e-4, 1e-5],
        "clf__alpha": [0.5, 1.0]
    }
}

# Set up Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=420)

# Define the best Dimensionality Reduction Technique from Method 3
pca = PCA(n_components=5)

# Store results
method_1_results = {}

for model_name, model in models.items():
    print(f"\rEvaluating {model_name}...", end="", flush=True)
    
    # Combine the dimensionality reduction, and model into a pipeline
    pipeline = Pipeline(
        steps = [
            ("preprocessor", preprocessor),
            ("pca", pca),
            ("clf", model),
        ])
    
    # Perform hyperparameter tuning with GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=skf, scoring="roc_auc")
    if model_name in ["KNN", "FNN"]:
        grid_search.fit(X_train, y_train)
    else:
        grid_search.fit(X_train, y_train, clf__sample_weight=sample_weights)
    
    # Evaluate the model on the test set
    best_pipeline = grid_search.best_estimator_
    
    y_pred = best_pipeline.predict(X_test)
    y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    metric_results = {}
    for metric_name, metric_func in metrics.items():
        if metric_name == "ROC AUC":
            metric_results[metric_name] = metric_func(y_test, y_pred_proba)
        elif metric_name == "Specificity":
            metric_results[metric_name] = metric_func(y_test, y_pred, pos_label=0)
        else:
            metric_results[metric_name] = metric_func(y_test, y_pred)
    
    # Store results
    method_1_results[model_name] = {
        "best Clf": best_pipeline,
        "Best Parameters": grid_search.best_params_,
        "Best ROC-AUC Score": grid_search.best_score_,
        "Metrics": metric_results
    }
    print(f"\rEvaluation completed for {model_name}")

print("All Evaluation completed.")

Evaluation completed for RF
Evaluation completed for GB
Evaluation completed for SVM
Evaluation completed for LR
Evaluation completed for KNN
Evaluation completed for FNN
All Evaluation completed.


In [6]:
# Print results
for model_name, result in method_1_results.items():
    print(f"\n{model_name} Results:")
    print(f"Best Parameters: {result['Best Parameters']}")
    print(f"Best ROC-AUC Score: {result['Best ROC-AUC Score']}")


RF Results:
Best Parameters: {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 4, 'clf__min_samples_split': 6, 'clf__n_estimators': 403}
Best ROC-AUC Score: 0.9554166666666667

GB Results:
Best Parameters: {'clf__criterion': 'squared_error', 'clf__learning_rate': 0.35, 'clf__loss': 'exponential', 'clf__max_depth': 6, 'clf__n_estimators': 150, 'clf__tol': 1e-12}
Best ROC-AUC Score: 0.9517658730158731

SVM Results:
Best Parameters: {'clf__C': 0.1, 'clf__kernel': 'sigmoid', 'clf__tol': 0.001}
Best ROC-AUC Score: 0.9520039682539683

LR Results:
Best Parameters: {'clf__C': 0.36, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
Best ROC-AUC Score: 0.9482539682539682

KNN Results:
Best Parameters: {'clf__n_neighbors': 17, 'clf__p': 2, 'clf__weights': 'distance'}
Best ROC-AUC Score: 0.9439781746031747

FNN Results:
Best Parameters: {'clf__activation': 'relu', 'clf__alpha': 1.0, 'clf__early_stopping': True, 'clf__hidden_layer_sizes': (

In [7]:
performance_metrics = {metric_name: [] for metric_name in metrics.keys()}

for model_name, result in method_1_results.items():
    # print(model_name)
    for metric_name, score in result["Metrics"].items():
        # print(metric_name, ":", score)
        performance_metrics[metric_name].append(score)

df_performance = pd.DataFrame(performance_metrics)
df_performance.insert(0, "Models", [model_name for model_name in models])
df_performance = df_performance.round(3)
df_performance

,Models,Balanced Accuracy,F1 Score,ROC AUC,Sensitivity,Specificity,Precision
0,RF,0.914,0.866,0.978,0.935,0.892,0.806
1,GB,0.834,0.769,0.946,0.806,0.862,0.735
2,SVM,0.922,0.879,0.992,0.935,0.908,0.829
3,LR,0.938,0.896,0.992,0.968,0.908,0.833
4,KNN,0.912,0.885,0.985,0.871,0.954,0.900
5,FNN,0.922,0.879,0.975,0.935,0.908,0.829


# t-SVD-Based Classification Pipelines 

In [8]:
# These hyperparameters were tested before fine-tuning each one at finer granularity to improve model performance

param_grids = {
    "RF": {
        "clf__n_estimators": [300, 400],
        "clf__max_depth": [None],
        "clf__min_samples_split": [5, 6],
        "clf__min_samples_leaf": [2, 3],
        "clf__criterion": ["gini", "entropy", "log_loss"],
#         "clf__class_weight": [{0: 1, 1: 3}],
        "clf__max_features": ["log2"], 
    },
    "GB": {
        "clf__n_estimators": [120, 125, 150],
        "clf__loss": ["log_loss", "exponential"],
        "clf__learning_rate": [0.3, 0.35],
        "clf__max_depth": [3, 4, 5],
        "clf__criterion": ["friedman_mse", "squared_error"],
        "clf__tol": [1e-10, 1e-11]
    },
    "SVM": {
        "clf__C": [0.15, 0.25],
        "clf__kernel": [ "rbf", "sigmoid"],
        "clf__tol": [1e-2, 1e-3],
    },
    "LR": {
        "clf__C": [0.1, 0.2, 0.25],
        "clf__penalty": ["l2"],
        "clf__solver": ["lbfgs", "liblinear", "saga", "sag"],
        "clf__tol": [1e-3, 1e-6, 1e-8],
        "clf__max_iter": [5000, 7000],
    },
    "KNN": {
        "clf__n_neighbors": [17, 18],
        "clf__weights": ["distance"],
        "clf__p": [3, 4, 5, 6]
    },
    "FNN": {
        "clf__hidden_layer_sizes": [(125, 155), (150,)],
        "clf__max_iter": [15000, 15050],
        "clf__activation": ["relu", "logistic", "identity"], 
        "clf__solver": ["adam", "sgd"],
        "clf__learning_rate": ["adaptive", "constant"],
        "clf__early_stopping": [True],
        "clf__tol": [1e-2, 1e-3],
        "clf__alpha": [0.01, 1.0]
    }
}

# Set up Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=420)

# Define the best Dimensionality Reduction Technique from Method 3
t_svd = TruncatedSVD(n_components=5)

# Store results
method_2_results = {}

for model_name, model in models.items():
    print(f"\rEvaluating {model_name}...", end="", flush=True)
    
    # Combine the preprocessing, dimensionality reduction, and model into a pipeline
    pipeline = Pipeline(
        steps = [
            ("preprocessor", preprocessor),
            ("t-svd", t_svd),
            ("clf", model),
        ])
    
    # Perform hyperparameter tuning with GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=skf, scoring="roc_auc")
    if model_name in ["KNN", "FNN"]:
        grid_search.fit(X_train, y_train)
    else:
        grid_search.fit(X_train, y_train, clf__sample_weight=sample_weights)
    
    # Evaluate the model on the test set
    best_pipeline = grid_search.best_estimator_
    y_pred = best_pipeline.predict(X_test)
    y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    metric_results = {}
    for metric_name, metric_func in metrics.items():
        if metric_name == "ROC AUC":
            metric_results[metric_name] = metric_func(y_test, y_pred_proba)
        elif metric_name == "Specificity":
            metric_results[metric_name] = metric_func(y_test, y_pred, pos_label=0)
        else:
            metric_results[metric_name] = metric_func(y_test, y_pred)
    
    # Store results
    method_2_results[model_name] = {
        "best Clf": best_pipeline,
        "Best Parameters": grid_search.best_params_,
        "Best ROC-AUC Score": grid_search.best_score_,
        "Metrics": metric_results
    }
    print(f"\rEvaluation completed for {model_name}")

print("Evaluation completed.")

Evaluation completed for RF
Evaluation completed for GB
Evaluation completed for SVM
Evaluation completed for LR
Evaluation completed for KNN
Evaluation completed for FNN
Evaluation completed.


In [9]:
# Print results
for model_name, result in method_2_results.items():
    print(f"\n{model_name} Results:")
    print(f"Best Parameters: {result['Best Parameters']}")
    print(f"Best ROC-AUC Score: {result['Best ROC-AUC Score']}")


RF Results:
Best Parameters: {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 3, 'clf__min_samples_split': 6, 'clf__n_estimators': 300}
Best ROC-AUC Score: 0.9331469699890753

GB Results:
Best Parameters: {'clf__criterion': 'friedman_mse', 'clf__learning_rate': 0.3, 'clf__loss': 'log_loss', 'clf__max_depth': 4, 'clf__n_estimators': 125, 'clf__tol': 1e-11}
Best ROC-AUC Score: 0.9345286292654714

SVM Results:
Best Parameters: {'clf__C': 0.25, 'clf__kernel': 'sigmoid', 'clf__tol': 0.001}
Best ROC-AUC Score: 0.9465587044534413

LR Results:
Best Parameters: {'clf__C': 0.1, 'clf__max_iter': 5000, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'clf__tol': 0.001}
Best ROC-AUC Score: 0.9451609793715057

KNN Results:
Best Parameters: {'clf__n_neighbors': 18, 'clf__p': 4, 'clf__weights': 'distance'}
Best ROC-AUC Score: 0.932494698284172

FNN Results:
Best Parameters: {'clf__activation': 'identity', 'clf__alpha': 0.01, 'clf__early_stop

In [10]:
performance_metrics = {metric_name: [] for metric_name in metrics.keys()}

for model_name, result in method_2_results.items():
    # print(model_name)
    for metric_name, score in result["Metrics"].items():
        # print(metric_name, ":", score)
        performance_metrics[metric_name].append(score)

df_performance = pd.DataFrame(performance_metrics)
df_performance.insert(0, "Models", [model_name for model_name in models])
df_performance = df_performance.round(3)
df_performance

,Models,Balanced Accuracy,F1 Score,ROC AUC,Sensitivity,Specificity,Precision
0,RF,0.905,0.862,0.983,0.903,0.908,0.824
1,GB,0.873,0.825,0.966,0.839,0.908,0.812
2,SVM,0.929,0.892,0.991,0.935,0.923,0.853
3,LR,0.938,0.896,0.994,0.968,0.908,0.833
4,KNN,0.912,0.885,0.988,0.871,0.954,0.900
5,FNN,0.742,0.652,0.993,0.484,1.000,1.000
